# Модули и пакети

## Какво е модул?

Всеки един Python файл (.py) на практика е модул. Освен това е възможно библиотека, написана на С, и вмъкната динамично също да бъде модул. Третия тип модули са вградените в езика такива.

В тази лекция се фокусираме върху първия тип и модули и как можем да ги създаваме, вмъкваме и боравим с тях.

## Как да създам модул?

Казахме, че всеки Python файл е валиден модул. 

Нека създадем един такъв с няколко дефиниции вътре (в папката с тази тетрадка вече би трябвало да се съдържа файл `hitchhikers.py`).

## `import`

Имената, функциите и класовете, които създадохме в този файл, не могат да бъдат достъпени директно от друг файл:

In [1]:
compute()

NameError: name 'compute' is not defined

Можем обаче да ги вмъкнем в друг файл (модул) чрез `import {името_на_модула}` (името на файла преди разширението `.py` се превръща в име на модула):

In [2]:
import hitchhikers

hitchhikers.compute()

Hm, I'll have to think about that. Return to this place in exactly 7.5 million years...


42

`import` освен, че интерпретира целия код на модула, добавя имената и дефинициите в един обект от тип модул, имащ името на модула. Затова и ги достъпваме чрез `името_на_модула.име_на_обекта`.

Какво се съдържа в един модул можем лесно да видим с `dir()`:

In [3]:
dir(hitchhikers)

['ANSWER',
 'TheGreatDeepThought',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'compute']

In [4]:
hitchhikers.ANSWER

42

In [5]:
computer = hitchhikers.TheGreatDeepThought()
computer.ask()

Shush! The show is back on.


In [6]:
hitchhikers.__name__

'hitchhikers'

In [7]:
dir()  # by default it shows the contents of the *current* module

['In',
 'Out',
 '_',
 '_2',
 '_3',
 '_4',
 '_6',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'computer',
 'exit',
 'get_ipython',
 'hitchhikers',
 'os',
 'quit',
 'sys']

In [8]:
__name__

'__main__'

## Добре, обаче `import` къде точно търси?

1. Директорията, в която се намира Python скрипта, който се изпълнява (или текущата, ако интерпретаторът е пуснат интерактивно)
2. Директориите, които са описани в `PYTHONPATH` променливата на средата
3. Лист от директории, зададен по време на инсталацията на Python

Този списък от възможни директории може да се види със `sys.path`:

In [9]:
import sys
sys.path

['/Users/alexander.ignatov/Documents/PythonCourse2022/13 - Modules',
 '/Users/alexander.ignatov/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles',
 '/Users/alexander.ignatov/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles/lib/python',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/alexander.ignatov/Library/Python/3.10/lib/python/site-packages',
 '/opt/homebrew/lib/python3.10/site-packages',
 '/opt/homebrew/Cellar/pygments/2.13.0_1/libexec/lib/python3.10/site-packages']

## Варианти на `import`

С `from {module} import {something}, {something_else}, ...` можем да импортираме само определени имена от модула, като те биват добавени към съдържанието на текущия (т.е. достъпваме ги без името на оригиналния модул и точка отпред):

In [10]:
from hitchhikers import compute
compute()

Hm, I'll have to think about that. Return to this place in exactly 7.5 million years...


42

С `from {module} import {something} as {alias}, {something_else} as {other_alias}, ...` можем да прекръстим импортираните имена:

In [11]:
from hitchhikers import ANSWER, TheGreatDeepThought as Computer
comp = Computer()
comp.ask() == ANSWER

Shush! The show is back on.


False

In [12]:
"ANSWER" in dir()

True

Ако искаме абсолютно всички имена на вмъкнем и ползваме в текущия модул по този начин (без тези, започващи с подчертавка `_`), можем да използваме астериск `*`:

In [13]:
# изпълни тази клетка ако си изпълнил горните, за да се зачистят import-ите
del hitchhikers, ANSWER, compute, Computer

In [14]:
from hitchhikers import *

compute() == ANSWER

Hm, I'll have to think about that. Return to this place in exactly 7.5 million years...


True

Лимитация на астерикс синтаксиса е, че не може да използва в блок (може само на най-външното ниво на модула):

In [15]:
del compute, ANSWER, TheGreatDeepThought

In [16]:
def obtain_answer():
    from hitchhikers import *  # 💥
    return compute()

SyntaxError: import * only allowed at module level (3855291323.py, line 2)

Както казахме, по подразбиране from {module} import * вмъква абсолютно всички имена от `module`, които не започват с подчертавка. Имаме всъщност контрол над това, кое може да се вмъкне чрез астерикс, като дефинираме `__all__` във въпросния модул. Стойността му е лист от всички имена, които ще бъдат вмъкнати от `*`.

*Пример*:

След добавяне на
```python
__all__ = ['compute', 'TheGreatDeepThought']
```
в `hitchhikers.py`, следният код, изпълнен в `script.py` (в същата директория) ще хвърли `NameError`:
```python
from hitchhikers import *
print(hitchhikers.ANSWER)  # 💥
```

## Пакети

Пакет e набор от модули. За Python всяка директория, в която има модули, се превръща в пакет (package).

*Note*: Във версии по-ранни от Python 3.3 трябва задължително в директорията да има файл с име `__init__.py`.

В директорията на тетрадката би трябвало да има папка `game`, съдържаща няколко файла и папки:

In [17]:
!tree game

game
├── engine.py
├── level.py
├── player.py
└── players
    ├── ai.py
    ├── input_player.py
    └── mock_player.py

1 directory, 6 files


В горния пример `game` е пакет, съдържащ модулите `engine`, `level` и `player`. Освен тях, той съдържа и подпакетът `players`.

Ако искаме да вмъкнем някой модул от пакета, можем да го направим чрез името на пакета (или всички пакети по веригата, разделени с точка), последвано от точка и името на въпросния модул:

In [18]:
import game.level

game.level.EASY

Level(word='SCRIPT', failed_attempts=10)

In [19]:
import game.players.ai

game.players.ai.AI(10)

Вече въведените по-горе синтактични варианти на `import` също важат:

In [20]:
from game.players.ai import AI
from game.level import EASY as easy, MEDIUM as medium, HARD as hard
from game.engine import *

Освен това, можем и да вмъкнем модули чрез `from {package} import {module} [as {alias}], ...`:

In [21]:
from game import level, engine

print(level.EASY)
print(engine.GameState)

Level(word='SCRIPT', failed_attempts=10)
<enum 'GameState'>


## `__init__.py`

На теория можем и да импортнем само пакета. По подразбиране това няма да добави нови модули и имена:

In [22]:
del game.level, game.player, game.engine  # зачисти тетрадката от предните импорти

In [23]:
import game

game.level  # 💥

AttributeError: module 'game' has no attribute 'level'

Ако искаме да добавим и модули от пакета при импортирането му, можем да ги импортнем в `__init__.py`, намиращ се в директорията на пакета.

Т.е. ако в `game/__init__.py` имаме:
```python
import game.engine, game.level, game.player
```

то можем да импортнем пакета `game` и да използваме всички модули от него:
```python
# в друг файл, извън пакета `game`:
import game
print(game.level.EASY)  # no error
```

В `__init__.py` можем да напишем какъвто искаме инициализационен код, глобален за всички модули в пакета. Съдържанието на скрипта се изпълнява веднага при импортиране на пакета.

Както при модулите, така и тука можем да дефинираме поведението на `from {package} import *` чрез `__all__`. По подразбиране, както видяхме за `import {package}`, това е празен списък, т.е. нищо няма да се вмъкне (за разлика от поведението при модулите, когато се вмъква абсолютно всяко име от модула, което не започва с подчертавка).

Т.е. ако напишем в `game/__init__.py`:

```python
__all__ = ["engine", "level", "player"]
```

то ще можем:

```python
# в друг файл, извън пакета `game`:
from game import *
print(level.EASY)  # no error
```

## Релативни импорти

Дотук разгледахме примерни за **абсолютни** импорти, т.е. достъпът до даден модул от рамките на пакета или извън него става през пътя от пакета до модула, например `game.players.ai` достъпва модулът `ai` от пакета `players` в пакета `game`.


In [24]:
import game.players.ai

Със значението на `.` и `..` от Unix файловата система, можем да използваме същите тези символи за **релативни** импорти в Python. Те се оценяват спрямо локацията на `import` statement-a.

Например, във файла `game/players/input_player.py` ни трябва `player` модула от пакета `game`. Можем да го направим по абсолютен и релативен начин:

```python
from game import player  # абсолютен импорт
```

```python
from .. import player  # релативен импорт
```

* `..` означава "пакетът, намиращ се над текущия".
* `..pkg` означва модулът/пакетът `pkg` от пакетът, намиращ се над текущия.

Например:
```python
from ..player import Player
```
Ще вмъкне името `Player` от модула `player` от пакета, намиращ се над текущия.

* `.` означава "текущия пакет".
* `.pkg` означава модулът/пакетът `pkg` от текущия пакет.

Релативните импорти имат недостатъка обаче, че зависят от местоположението на `import`-a. Освен това в скриптове (т.е. изпълним код, който не е вмъкнат чрез модул) имат различно поведение:

In [25]:
from . import hitchhikers

ImportError: attempted relative import with no known parent package

## `if __name__ == "__main__"`

Както бяхме споменали, при импорт се изпълнява кода на съответния модул. Като пример за това можем да изведем философията на Python, намираща се във вградения модул `this`:

In [26]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Текущо-изпълнимият файл/модул/скрипт за Python се казва `"__main__"`, т.е. неговия `__name__` е `"__main__"`:
```python

In [27]:
__name__

'__main__'

Ако файлът не се изпълнява директно, а бъде импортнат от друг, то в неговия `__name__` ще е името на модула. Това означава, че можем да различим дали файлът се изпълнява директно или е импортнат. 

Полезно е в случаите, когато искаме да напишем примерно някакви тестове или демонстрации на модула, които да се изпълнят само ако го изпълним директно, и да не се изпълняват при всяко вмъкване. . В такива случаи използваме `if __name__ == "__main__": ...` (разгледайте например `game/engine.py`).

## Управление на пакети

### Какво е `pip`?

Пакетите се създават с цел лесно преизползване. При използване на външни пакети в проекта се появяват конкретни проблеми, които трябва да се решават - как да се инсталират, как да се обновят, как да се изтрият, как да се решават зависимостите и т.н. С това ни помагат различните "package manager"-и, като python-ският такъв е `pip` (името му е рекурсивен акроним: "**P**IP **I**nstalls **P**ackages").

### Къде е `pip`?

Управлението на пакетите е важна част от разработката и затова от Python 3.4 и 2.7.9 насам `pip` е част от инсталацията на Python 3 и Python 2 респективно.

Можем да проверим дали PIP е инсталиран като се опитаме да видим локацията на `pip3` командата като изпълним `which pip3` (или `where pip3` под Windows):

```bash

In [1]:
!which pip3  # linux / macOS

/opt/homebrew/bin/pip3


В случай, че няма `pip3`, е възможно да съществува само `pip` командата:

In [3]:
!which pip

/opt/homebrew/bin/pip


В случай, че имаме и двете команди, можем да ги сраним като видим разликите във версията (ако са еднакви, то няма значение дали използваме `pip` или `pip3`. Оттук нататък до края на тетрадката ще ги считаме за едни и същи):

In [4]:
!pip3 --version && pip --version

pip 22.3.1 from /opt/homebrew/lib/python3.10/site-packages/pip (python 3.10)
pip 22.3.1 from /opt/homebrew/lib/python3.10/site-packages/pip (python 3.10)


### Как да преинсталирам `pip` ако нещо не е наред?

В случай, че `pip` не може да бъде намерен, има два варианта:

1. `pip` е инсталиран, но пътя до него не е в `$PATH` променливата на средата
2. `pip` не е инсталиран. Тогава можем да го сложим по два начина:
    1. Инсталираме `pip` от [`get-pip.py`](https://github.com/pypa/get-pip) скрипта
    2. Инсталираме `pip` чрез [`ensurepip`](https://docs.python.org/3/library/ensurepip.html#module-ensurepip) модула

In [6]:
!python3 -m ensurepip --upgrade  # за windows е `python` вместо `python3`

Looking in links: /var/folders/q1/7m4c3ff153j93q271xrs9y5r0000gq/T/tmpkkspnayv


***NOTE***: `ensurepip` не тегли от интернет нищо - директно инсталира версията на `pip`, която е bundle-ната със съответната версия на Python. В случай, че искаме по-нова от дадената, трябва след това ръчно да актуализираме `pip` чрез `python3 -m pip install --upgrade pip` или `pip3 install --upgrade pip` (респективно само `python` вместо `python3` под Windows).

### Какви подкоманди има `pip`?

Пакети се инсталират с `pip install <package1_name> [<package2_name> ...]`:

In [15]:
!pip install requests

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)


Пакетите по подразбиране се търсят в [PyPI](https://pypi.org/) (чете се "пай пи ай", а не "пипи"/"пайпи"/"пайпай"), който е Python Package Index. Това е публичен индекс от пакети, към който всеки потребител може да добавя, допринася, търси и ползва.

Ако искаме `pip` да търси в друг индекс (примерно такъв с частни репозиторита), можем да го променим чрез `-i [index_url]` аргумент към `pip install`. Повече за това [тук](https://realpython.com/what-is-pip/#using-a-custom-package-index).

Информация за инсталиран пакет може да изведем с `pip show <package_name>`:

In [16]:
!pip show requests

Name: requests
Version: 2.28.1
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: /opt/homebrew/lib/python3.10/site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: nbdime, requests-threads, Sphinx


Деинсталирането пък съответно става по същия начин, но този път с подкомандата `uninstall`:

In [9]:
!pip uninstall requests <<< "y"

Found existing installation: requests 2.28.1
Uninstalling requests-2.28.1:
  Would remove:
    /opt/homebrew/lib/python3.10/site-packages/requests-2.28.1.dist-info/*
    /opt/homebrew/lib/python3.10/site-packages/requests/*
Proceed (Y/n)?   Successfully uninstalled requests-2.28.1


Всички инсталирани пакети и техните версии може да видим с `pip list`:

In [13]:
!pip list

Package                       Version
----------------------------- -----------
alabaster                     0.7.12
anyio                         3.6.2
appnope                       0.1.3
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
astroid                       2.12.12
asttokens                     2.0.8
attrs                         21.4.0
Automat                       22.10.0
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.11.1
bleach                        5.0.1
certifi                       2022.9.24
cffi                          1.15.1
charset-normalizer            2.1.1
click                         8.1.3
colorama                      0.4.6
constantly                    15.1.0
debugpy                       1.6.3
decorator                     5.1.1
defusedxml                    0.7.1
dill                          0.3.6
docutils                      0.17.1
entrypoints                   0.4
execu

Извеждането на списъка със зависимости (инсталирани пакети + версия) в "requirements" формат може да се извърши с `pip freeze`:

In [14]:
!pip freeze

alabaster==0.7.12
anyio==3.6.2
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
astroid==2.12.12
asttokens==2.0.8
attrs==21.4.0
Automat==22.10.0
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==5.0.1
certifi==2022.9.24
cffi==1.15.1
charset-normalizer==2.1.1
click==8.1.3
colorama==0.4.6
constantly==15.1.0
debugpy==1.6.3
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.6
docutils==0.17.1
entrypoints==0.4
executing==1.1.0
fastjsonschema==2.16.2
gitdb==4.0.10
GitPython==3.1.29
hyperlink==21.0.0
idna==3.4
imagesize==1.4.1
importlib-metadata==5.1.0
incremental==22.10.0
ipykernel==6.16.0
ipython==8.5.0
ipython-genutils==0.2.0
ipywidgets==7.7.2
isort==5.10.1
jedi==0.18.1
Jinja2==3.1.2
jsonschema==3.2.0
jupyter-book==0.13.1
jupyter-cache==0.4.3
jupyter-events==0.4.0
jupyter-server-mathjax==0.2.6
jupyter-sphinx==0.3.2
jupyter_client==7.4.8
jupyter_core==5.1.0
jupyter_server==2.0.1
jupyter_server_terminals==0.4.2
jupyterlab-pygments==0.2.2
jupyterlab-widgets==1.1.1
latexcod

Обикновено изхода от командата се запазва във файл, наречен `requirements.txt`. Повече за това по-надолу.

## Виртуални среди и `venv`

### Защо?

Когато инсталираме пакет, той след това може да бъде използван от всички Python проекти на машината (или на потребителя). Това обаче може да доведе до конфликти. Да предположим, че имаме един проект, който изисква например пакетът `А` да бъде с версия $ \geq X $, докато друг да е направен да работи с версия на `A`, която да е по-малка от $ X $ (т.е. ъпдейтването до версия $ X $ би счупило проекта). Инсталацията на пакетът `A` по познатия начин обаче е глобална и не върши работа в случая - трябва ни някакъв начин, по който да имаме различни инсталации на пакета за различните проекти.

Това е идеята на т.нар. "virtual environments" - създават виртуална среда, която да се отнася само за конкретен проект, в която той да се конфигурира, да се изтеглят пакетите, от които зависи и т.н.

### Как?

С вградената билбиотека `venv` създаваме virtual environment. Изпълняваме я като модул (с флаг `-m`) и като параметър указваме името на виртуалната среда, която ще бъде създадена в текущата директория:

In [1]:
!python3 -m venv venv

***Note 1***: Обикновено се кръщава също `venv`.

***Note 2***: Директорията (`venv` в този случай) на виртуалната среда трябва да бъде игнорирана от Git (т.е. да ѝ се добави името на нов ред в `.gitignore` файла).

Предната команда ще създаде в текущата директория папката `venv`, в която се намира всичко необходимо на виртуалната среда, за да работи. Тя обаче още няма да е активирана, като това става чрез изпълняване веднъж на:

In [2]:
!source venv/bin/activate

***Note***: на Windows ще е `venv\Scripts\activate.bat` (или `venv\Scripts\activate.ps1` на PowerShell)

Това ще пренасочи команди като `python`/`python3` и `pip`/`pip3` към локалните копия, намиращи се под директорията на виртуалната среда. Инсталираните пакети също отиват там. По подразбиране няма такива (освен самите `pip` и `setuptools`):

In [5]:
!source venv/bin/activate && pip list  # активираме пак понеже jupyter клетките не запазват bash сесиите

Package    Version
---------- -------
pip        22.2.2
setuptools 65.4.1

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


Работата във виртуалната среда приключва, когато приключи текущата конзолна сесия или когато бъде изпълнена командата `deactivate` (`venv\Scripts\deactivate.bat` под Windows).

## `requirements.txt`

Ако свалим даден проект локално и се опитаме да го изпълним, то ако той зависи от някакви third-party модули/пакети/билбиотеки, ще ни хвърли `ImportError`, понеже няма да ги намери. Трудно и излишно времеемко е обаче ръчно да проверим кои зависимости ги няма и да ги свалим. Затова ни служи `requirements.txt` - файл, в който всички dependency-та са описани (име на пакет и версии).

Създава се чрез изхода на `pip freeze`:

In [6]:
!pip freeze > requirements.txt

Съдържанието е във формат `{има на пакет}{знак за сравнение}{версия}`:

In [7]:
!cat requirements.txt

alabaster==0.7.12
anyio==3.6.2
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
astroid==2.12.12
asttokens==2.0.8
attrs==21.4.0
Automat==22.10.0
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==5.0.1
certifi==2022.9.24
cffi==1.15.1
charset-normalizer==2.1.1
click==8.1.3
colorama==0.4.6
constantly==15.1.0
debugpy==1.6.3
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.6
docutils==0.17.1
entrypoints==0.4
executing==1.1.0
fastjsonschema==2.16.2
gitdb==4.0.10
GitPython==3.1.29
hyperlink==21.0.0
idna==3.4
imagesize==1.4.1
importlib-metadata==5.1.0
incremental==22.10.0
ipykernel==6.16.0
ipython==8.5.0
ipython-genutils==0.2.0
ipywidgets==7.7.2
isort==5.10.1
jedi==0.18.1
Jinja2==3.1.2
jsonschema==3.2.0
jupyter-book==0.13.1
jupyter-cache==0.4.3
jupyter-events==0.4.0
jupyter-server-mathjax==0.2.6
jupyter-sphinx==0.3.2
jupyter_client==7.4.8
jupyter_core==5.1.0
jupyter_server==2.0.1
jupyter_server_terminals==0.4.2
jupyterlab-pygments==0.2.2
jupyterlab-widgets==1.1.1
latexcod

Използването на файла (т.е. изтеглянето на всички правилни версии на описаните пакети) става чрез аргумента `-r` на `pip install`:

In [8]:
!pip install -r requirements.txt

## Да обобщим: setup на проект for dummies

1. Създаване на виртуална среда:

    `python3 -m venv venv`

    *Важно:* при използване на система за контрол на версиите (напр. Git) трябва новосъздадената директория да бъде игнорирана от нея (`>> .gitignore`).
2. Активиране на средата:

    `source venv/bin/activate` (unix) или `venv\Scripts\activate.bat` (windows)

3. Подсигуряване на това, че `pip` е последна версия:

    `pip install --upgrade pip`

3. Работата по инсталиране на пакети, пускане на кода и т.н. трябва задължително да става докато е активирана виртуалната среда

    ...

4. След инсталиране на всеки пакет трябва да се обновява списъкът със зависимости:

    `pip freeze > requirements.txt`


